# 📊 MGMT 467 - Unit 2 Lab 2: Churn Modeling with BigQueryML + Feature Engineering
**Date:** 2025-10-16

In this lab you will:
- Connect to BigQuery from Colab
- Create features and labels
- Engineer new features from user behavior
- Train and evaluate logistic regression models
- Reflect on modeling assumptions and interpret results

In [1]:
# ✅ Authenticate and set up GCP project
from google.colab import auth
auth.authenticate_user()

project_id = "mgmt-467-4677"  # <-- Replace with your actual project ID
!gcloud config set project $project_id

Updated property [core/project].


In [2]:
# ✅ Verify BigQuery access
%%bigquery --project $project_id
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user

Query is running:   0%|          |

Downloading:   0%|          |

,today,user
0,2025-10-26,lencalpha@gmail.com


In [10]:
# ✅ Prepare base churn features
%%bigquery --project $project_id
CREATE OR REPLACE TABLE `netflix` AS
SELECT
  user_id,
  region,
  plan_tier,
  age_band,
  avg_rating,
  total_minutes,
  avg_progress,
  num_sessions,
  churn_label
FROM `netflix.cleaned_features`
WHERE churn_label IS NOT NULL;

Executing query with job ID: 5f893c8b-2876-4190-950c-baa21b2802f4
Query executing: 0.26s


ERROR:
 400 Table "netflix" must be qualified with a dataset (e.g. dataset.table).; reason: invalid, location: netflix, message: Table "netflix" must be qualified with a dataset (e.g. dataset.table).

Location: US
Job ID: 5f893c8b-2876-4190-950c-baa21b2802f4



In [4]:
# ✅ Train base logistic regression model
%%bigquery --project $project_id
CREATE OR REPLACE MODEL `your_dataset.churn_model`
OPTIONS(model_type='logistic_reg') AS
SELECT
  region,
  plan_tier,
  age_band,
  avg_rating,
  total_minutes,
  avg_progress,
  num_sessions,
  churn_label
FROM `your_dataset.churn_features`;

Executing query with job ID: 0d910539-f061-42ef-aade-eaac7db29410
Query executing: 0.23s


ERROR:
 404 Not found: Dataset mgmt-467-4677:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt-467-4677:your_dataset was not found in location US

Location: US
Job ID: 0d910539-f061-42ef-aade-eaac7db29410



In [5]:
# ✅ Evaluate base model
%%bigquery --project $project_id
SELECT *
FROM ML.EVALUATE(MODEL `your_dataset.churn_model`);

Executing query with job ID: 6823eb0a-1070-400d-8065-d30efc848c71
Query executing: 0.57s


ERROR:
 404 Not found: Dataset mgmt-467-4677:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt-467-4677:your_dataset was not found in location US

Location: US
Job ID: 6823eb0a-1070-400d-8065-d30efc848c71



In [6]:
# ✅ Predict churn with base model
%%bigquery --project $project_id
SELECT
  user_id,
  predicted_churn_label,
  predicted_churn_label_probs
FROM ML.PREDICT(MODEL `your_dataset.churn_model`,
                (SELECT * FROM `your_dataset.churn_features`));

Executing query with job ID: 39c5b3b0-24ae-48cb-8673-a0b4d7f9f055
Query executing: 0.37s


ERROR:
 404 Not found: Dataset mgmt-467-4677:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt-467-4677:your_dataset was not found in location US

Location: US
Job ID: 39c5b3b0-24ae-48cb-8673-a0b4d7f9f055




## 🛠️ Feature Engineering Section

We will now engineer new features to improve model performance:

- Bucket continuous variables
- Create interaction terms
- Add behavioral flags


In [7]:

# ✅ Create enhanced feature set
%%bigquery --project $project_id
CREATE OR REPLACE TABLE `your_dataset.churn_features_enhanced` AS
SELECT
  user_id,
  region,
  plan_tier,
  age_band,
  avg_rating,
  total_minutes,
  CASE
    WHEN total_minutes < 100 THEN 'low'
    WHEN total_minutes BETWEEN 100 AND 300 THEN 'medium'
    ELSE 'high'
  END AS watch_time_bucket,
  avg_progress,
  num_sessions,
  CONCAT(plan_tier, '_', region) AS plan_region_combo,
  IF(total_minutes > 500, 1, 0) AS flag_binge,
  churn_label
FROM `your_dataset.churn_features`;


Executing query with job ID: e9c9c19f-2e3f-4fa8-909b-5d2ceec30c98
Query executing: 0.39s


ERROR:
 404 Not found: Dataset mgmt-467-4677:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt-467-4677:your_dataset was not found in location US

Location: US
Job ID: e9c9c19f-2e3f-4fa8-909b-5d2ceec30c98



In [8]:

# ✅ Train enhanced model
%%bigquery --project $project_id
CREATE OR REPLACE MODEL `your_dataset.churn_model_enhanced`
OPTIONS(model_type='logistic_reg') AS
SELECT
  region,
  plan_tier,
  age_band,
  watch_time_bucket,
  avg_rating,
  avg_progress,
  num_sessions,
  plan_region_combo,
  flag_binge,
  churn_label
FROM `your_dataset.churn_features_enhanced`;


Executing query with job ID: 4e59dcfc-c151-42e9-bf1a-7cf81b31c78c
Query executing: 0.19s


ERROR:
 404 Not found: Dataset mgmt-467-4677:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt-467-4677:your_dataset was not found in location US

Location: US
Job ID: 4e59dcfc-c151-42e9-bf1a-7cf81b31c78c



In [9]:

# ✅ Evaluate enhanced model
%%bigquery --project $project_id
SELECT *
FROM ML.EVALUATE(MODEL `your_dataset.churn_model_enhanced`);


Executing query with job ID: 17886016-116b-4842-ae2f-ddd88d9b5d37
Query executing: 0.45s


ERROR:
 404 Not found: Dataset mgmt-467-4677:your_dataset was not found in location US; reason: notFound, message: Not found: Dataset mgmt-467-4677:your_dataset was not found in location US

Location: US
Job ID: 17886016-116b-4842-ae2f-ddd88d9b5d37




## 🤔 Chain-of-Thought Prompts: Feature Engineering

### 1. Why bucket continuous values like watch time?
- What patterns become clearer by using categories like "low", "medium", "high"?



### 2. What value do interaction terms (e.g., `plan_tier_region`) add?
- Could some plans behave differently in different regions?

### 3. What’s the purpose of binary flags like `flag_binge`?
- Can these capture unique behaviors not reflected in raw totals?

### 4. After evaluating the enhanced model:
- Which new features helped the most?
- Did any surprise you?

✍️ Write your responses in a text cell below or in a shared doc for discussion.


## 🤔 Chain-of-Thought Prompts: Feature Engineering

### 1. Why bucket continuous values like watch time?
- What patterns become clearer by using categories like "low", "medium", "high"?

What patterns become clearer by using categories like "low", "medium", "high"?
Bucketing continuous variables like watch time into categories such as "low," "medium," and "high" can help simplify the relationship between the feature and the target variable (churn). It can make patterns clearer by:

Reducing noise: Grouping data into buckets can smooth out minor variations in watch time that might not be significant individually.
Highlighting non-linear relationships: The relationship between watch time and churn might not be linear. Bucketing can capture these non-linearities by treating different ranges of watch time differently.
Improving interpretability: Categories like "low," "medium," and "high" are often easier to understand and communicate than raw continuous values.
Handling outliers: Extreme values in watch time can have less impact when they are grouped into a bucket.
For example, you might find that users with "low" watch time have a high churn rate, while those with "medium" watch time have a lower churn rate, and those with "high" watch time have a moderate churn rate. This non-linear pattern would be harder to see with a continuous watch time variable.

### 2. What value do interaction terms (e.g., `plan_tier_region`) add?
- Could some plans behave differently in different regions?

Interaction terms like `plan_tier_region` can add value by capturing how the effect of one feature depends on the value of another feature. In this case, it can show if the churn rate associated with a particular plan tier is different across different regions. For example, a premium plan might have a low churn rate in one region but a higher churn rate in another due to local competition or cultural factors. This interaction effect might not be apparent if you only look at `plan_tier` and `region` independently.

### 3. What’s the purpose of binary flags like `flag_binge`?
- Can these capture unique behaviors not reflected in raw totals?

Binary flags like `flag_binge` (indicating if a user has watched over 500 minutes) are useful for highlighting specific behaviors that might be strongly associated with churn, but are not simply a linear function of the raw total. For example, "binge-watching" might be a distinct behavior pattern that indicates a highly engaged user who is less likely to churn, or it could indicate a user who is consuming content rapidly and might soon exhaust available content, leading to churn. A simple total minutes watched might not capture this nuanced behavior as effectively as a dedicated flag.

### 4. After evaluating the enhanced model:
- Which new features helped the most?
- Did any surprise you?

✍️ Write your responses in a text cell below or in a shared doc for discussion.